## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yining,CN,29.0361,114.5636,71.80,82,24,2.08,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,67.03,88,100,8.99,moderate rain
2,2,Ushuaia,AR,-54.8000,-68.3000,40.62,75,75,2.30,broken clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,62.01,60,0,15.50,clear sky
4,4,Barrow,US,71.2906,-156.7887,35.62,87,90,13.80,drizzle


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yining,CN,29.0361,114.5636,71.80,82,24,2.08,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,67.03,88,100,8.99,moderate rain
5,5,Honiara,SB,-9.4333,159.9500,76.59,89,62,6.29,light rain
6,6,Cape Town,ZA,-33.9258,18.4232,72.12,68,0,3.00,clear sky
7,7,Alice Springs,AU,-23.7000,133.8833,67.55,94,90,10.36,light intensity shower rain
12,12,Hilo,US,19.7297,-155.0900,74.28,93,90,6.91,light rain
18,18,Angoche,MZ,-16.2325,39.9086,77.79,79,61,14.70,broken clouds
20,20,Kurchum,KZ,48.5722,83.6542,66.25,26,86,8.57,overcast clouds
21,21,Port-Gentil,GA,-0.7193,8.7815,78.06,81,81,14.25,broken clouds
23,23,Bethel,US,41.3712,-73.4140,71.96,90,90,1.01,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yining,CN,71.80,few clouds,29.0361,114.5636,
1,Rikitea,PF,67.03,moderate rain,-23.1203,-134.9692,
5,Honiara,SB,76.59,light rain,-9.4333,159.9500,
6,Cape Town,ZA,72.12,clear sky,-33.9258,18.4232,
7,Alice Springs,AU,67.55,light intensity shower rain,-23.7000,133.8833,
12,Hilo,US,74.28,light rain,19.7297,-155.0900,
18,Angoche,MZ,77.79,broken clouds,-16.2325,39.9086,
20,Kurchum,KZ,66.25,overcast clouds,48.5722,83.6542,
21,Port-Gentil,GA,78.06,broken clouds,-0.7193,8.7815,
23,Bethel,US,71.96,overcast clouds,41.3712,-73.4140,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yining,CN,71.80,few clouds,29.0361,114.5636,Xiushui Hotel
1,Rikitea,PF,67.03,moderate rain,-23.1203,-134.9692,People ThankYou
5,Honiara,SB,76.59,light rain,-9.4333,159.9500,Heritage Park Hotel
6,Cape Town,ZA,72.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
7,Alice Springs,AU,67.55,light intensity shower rain,-23.7000,133.8833,Desert Palms Alice Springs


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))